<code>plot_RMSE_all_days_ALL_YEAR.ipynb</code> plots RMSE as a function of lead time for ALL DAYS for all S2S models.

In [1]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from S2S_sea_ice_plotting import RMSE_plot

### Inputs:
* <code>model_names</code>: which models to include (helpful for debugging)
* <code>seas_str</code>: which season to plot (string) (for this notebook, we'll just do ALL)
* <code>seas_sel</code>: months to include (array) (all months for this notebook)
* <code>vrile_thresh</code>: threshold for VRILE calculation (string)
* <code>clim_freq</code>: lead time in days or weeks (string)
* <code>TO_PLOT</code>: are we plotting SIE or SIE anom?
* <code>obs_name</code>: which set of obs to use (string)
* <code>obs_name_plt</code>: name of obs (for plot, string)
* <code>ALL_DAYS</code>: boolean; if true, include VRILE and nonVRILE days; if false, just use nonVRILE days

In [6]:
ALL_REGIONS = False 
model_names = ['ecmwf','ukmo','metreofr','ncep','DAMPED_ANOMALY']
#model_names = ['DAMPED_ANOMALY']
seas_str = 'JFM'
seas_sel = [1,2,3,4,5,6,7,8,9,10,11,12]
vrile_thresh = '05'
clim_freq = 'WEEKLY'
TO_PLOT = 'SIE RMSE'
obs_name = 'NSIDC_0079'
obs_name_plt = 'NASA Bootstrap'
ALL_DAYS = True

### Steps
1. Load on RMSE spreadsheets for all days and VRILE days, and combine all models into one spreadsheet 
2. Plot RMSE vs lead time for specified season for desired regions. 

Load <code>.csv</code> files

In [7]:
d_SIE_all = pd.DataFrame()
fpath_load = '/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/COMMON_LAND_MASK/data/'
for i_model in model_names:
    # Damped anomaly has different filename
    if i_model == 'DAMPED_ANOMALY':
        fpath_all = fpath_load+'{model_name}_{obs_name}/OBS_{obs_name}/'.format(model_name=i_model,obs_name=obs_name)       
        fpath_all = fpath_all+'COMMON_RF/VRILE_vs_NOVRILE_RMSE_MAE_{model_name}_{obs_name}_months{seas_str}_VRILE{vrile_thresh}_model_clim_freq_{clim_freq}.csv'.format(model_name=i_model,
                                                                                obs_name=obs_name,seas_str=seas_str,
                                                                                            vrile_thresh=vrile_thresh,
                                                                                            clim_freq=clim_freq)
        d_SIE = pd.read_csv(fpath_all)
        d_SIE['Model Name'] = 'Damped Anomaly'
        d_SIE_all = d_SIE_all.append(d_SIE)
        
    else:
        # Load ALL DAYS
        fpath_all = fpath_load+'{model_name}/OBS_{obs_name}/COMMON_RF/VRILE_vs_NOVRILE_RMSE_MAE_{model_name}_months{seas_str}_VRILE{vrile_thresh}_model_clim_freq_{clim_freq}.csv'.format(model_name=i_model,
                                                                                                obs_name=obs_name,
                                                                                                seas_str=seas_str,
                                                                                                vrile_thresh=vrile_thresh,
                                                                                                clim_freq=clim_freq)
        d_SIE = pd.read_csv(fpath_all)
        # Because we spelled "meteofr" wrong
        if i_model == 'metreofr':
            d_SIE['Model Name'] = 'meteofr'
        else:
            d_SIE['Model Name'] = i_model
        # 
        d_SIE_all = d_SIE_all.append(d_SIE)
        
    

In [8]:
fpath_all

'/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/COMMON_LAND_MASK/data/DAMPED_ANOMALY_NSIDC_0079/OBS_NSIDC_0079/COMMON_RF/VRILE_vs_NOVRILE_RMSE_MAE_DAMPED_ANOMALY_NSIDC_0079_monthsJFM_VRILE05_model_clim_freq_WEEKLY.csv'

Trim regions

In [9]:
regions_all = d_SIE_all['region'].unique().tolist()
regions_drop = ['St John']#,'Hudson Bay','Canadian Islands','Bering','Sea of Okhotsk']
regions_sel = d_SIE_all[~d_SIE_all['region'].isin(regions_drop)]['region'].unique().tolist()
if ALL_REGIONS == True:
    regions_list = regions_sel
    ncols = 5
    reg_save_str = 'REG_ALL'
    fpath_save = '/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/COMMON_LAND_MASK/figures/RMSE_all_days/'
    ftype = 'pdf'
else:
    regions_list = ['panArctic','Barents Sea','East Greenland Sea','Central Arctic','Kara-Laptev Sea',
                     'East Siberian-Beaufort-Chukchi Sea','Bering']
    ncols = 2
    reg_save_str = 'REG_TRIM'
    fpath_save = '/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/COMMON_LAND_MASK/figures/RMSE_all_days/'
    ftype='pdf'
if not os.path.exists(fpath_save):
    os.makedirs(fpath_save)

In [10]:
d_SIE_all = d_SIE_all[~d_SIE_all['region'].isin(regions_drop)]

Shorten region names and replace to plot in desired order

In [11]:
d_SIE_all['region'] = d_SIE_all['region'].replace({'East Greenland Sea':'E. Greenland Sea',
    'East Siberian-Beaufort-Chukchi Sea':'E. Sib./Beauf./Chuk. Sea'})
#model_R2_select['Model Name'] = model_R2_select['Model Name'].replace({'metreofr':'meteofr'})
#
d_SIE_all['Model Name'] = d_SIE_all['Model Name'].replace({'ecmwf':'ECMWF','ukmo':'UKMO','meteofr':'METEOFR',
                                                          'ncep':'NCEP'})
regions_sort = ["panArctic","Central Arctic","E. Sib./Beauf./Chuk. Sea","Kara-Laptev Sea",
                "Barents Sea","E. Greenland Sea","Bering"]
d_SIE_all['region sort'] = pd.Categorical(d_SIE_all['region'],regions_sort)


Group by region, lead day, model, and type (all days or non-VRILE days only, based on <code>ALL_DAYS</code>)

In [12]:
d_SIE_all_gb = d_SIE_all.groupby(['region sort','Model Name','type','lead days']).mean()
if ALL_DAYS == False:
    d_SIE_all_gb = d_SIE_all_gb.xs(('no VRILE days'),level=2)
elif ALL_DAYS == True:
    d_SIE_all_gb = d_SIE_all_gb.mean(level=(0,1,3))


In [13]:
if TO_PLOT == 'SIE RMSE':
    title_str = 'Sea Ice Extent'
    save_str = 'SIE'
else:
    title_str = 'Anomalous Sea Ice Extent'
    save_str = 'SIE_anom'

Plot

In [ ]:
nrows = 4
ncols = 2
no_regions = len(regions_sort)
if TO_PLOT == 'SIE RMSE':
    vmax = [1.05,0.75,0.75,0.375,0.255,0.375,0.255]
else:
    vmax = [0.75,0.305,0.505,0.305,0.205,0.205,0.205]
vmin = 0
fig1,ax1 = plt.subplots(nrows,ncols,figsize=(20,20))
for i in range(0,no_regions):
    ax_sel = ax1.flat[i]
    #i_reg = np.floor(i/2).astype(int)
    i_reg = i
    reg_sel = regions_sort[i_reg]
    #if np.mod(i,2) == 0:
    data_sel = d_SIE_all_gb.xs((reg_sel)).reset_index()
    #elif np.mod(i,2) == 1:
     #   data_sel = d_SIE_all_gb2.xs((reg_sel)).reset_index()
    # Plot
    RMSE_plot(ax_sel,data_sel,reg_sel,TO_PLOT,i,vmin,vmax[i_reg],no_regions,ncols,nrows)
    #if i == len(no_regions):
    #ax_sel.legend(bbox_to_anchor=(1.025,1),loc=2,borderaxespad=0,fontsize=18)
fig1.delaxes(ax=ax1.flat[7])        
fig1.tight_layout()

if TO_PLOT == 'SIE RMSE':
    fig1.suptitle('Predictions of Sea Ice Extent using {obs_name}'.format(obs_name=obs_name_plt),fontsize=32,y=1.02)
else:
    fig1.suptitle('Predictions of Anomalous Sea Ice Extent using {obs_name}'.format(obs_name=obs_name_plt),fontsize=32,y=1.02)
if ALL_DAYS == True:
    fpath_save1 = fpath_save+'ALL_DAYS_{save_str}_{seas_str}_RMSE_only_{obs_name}.pdf'.format(save_str=save_str,seas_str=seas_str,
                                                                                   obs_name=obs_name)
elif ALL_DAYS == False:
    fpath_save1 = fpath_save+'noVRILE_DAYS_{save_str}_{seas_str}_RMSE_only_{obs_name}.pdf'.format(save_str=save_str,seas_str=seas_str,
                                                                                   obs_name=obs_name)
fig1.savefig(fpath_save1,format='pdf',bbox_inches='tight')  
print(fpath_save1)

In [ ]:
d_SIE_all_gb.xs('Kara-Laptev Sea')